<a href="https://colab.research.google.com/github/Distifiaoktari/klp-9-project-apm-kelas-B/blob/main/Rulebased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
# Using import to load a library module into a program’s memory.
import pandas as pd
# Reading persona.csv as pandas DataFrame
df = pd.read_csv('drive/MyDrive/APM (1)/persona.csv')
df


,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17
...,...,...,...,...,...
4995,29,android,female,bra,31
4996,29,android,female,bra,31
4997,29,android,female,bra,31
4998,39,android,female,bra,31


In [ ]:
# Analyze descriptive statistics
df.head() # Returns the first 5 rows for quickly checking data
df.shape  # Returns a dimensionality of the DataFrame
df.describe().T # Generates descriptive statistic
df.isnull().values.any() # Returns any value is missing in DataFrame
df.isnull().sum() #  Returns how many missing values exist in the DataFrame

PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64

In [ ]:
# Number of unique <SOURCE>
df["SOURCE"].nunique() # Count number of distinct SOURCE elements
df["SOURCE"].value_counts()# Returns counts of SOURCE rows 
df["COUNTRY"].value_counts() # Returns counts of COUNTRY rows

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [ ]:
# Country breakdown of income averages

df.groupby("COUNTRY")["PRICE"].agg({"mean"})

# Country and Source breakdown of income averages

df.groupby(["COUNTRY", 'SOURCE'])["PRICE"].mean()

COUNTRY  SOURCE 
bra      android    34.387029
         ios        34.222222
can      android    33.330709
         ios        33.951456
deu      android    33.869888
         ios        34.268817
fra      android    34.312500
         ios        32.776224
tur      android    36.229437
         ios        33.272727
usa      android    33.760357
         ios        34.371703
Name: PRICE, dtype: float64

In [ ]:
# Average income on the basis of variables,

agg_df = df.groupby(["COUNTRY", 'SOURCE', "SEX", "AGE"])["PRICE"].mean().sort_values(ascending=False)
agg_df.head()

COUNTRY  SOURCE   SEX     AGE
bra      android  male    46     59.0
usa      android  male    36     59.0
fra      android  female  24     59.0
usa      ios      male    32     54.0
deu      android  female  36     49.0
Name: PRICE, dtype: float64

In [ ]:
# Convert the index names to variable names

agg_df = agg_df.reset_index() # If False, return a copy. Otherwise, do operation inplace and return None.
agg_df.head() # Lets look at new index numbers

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [ ]:
# Convert AGE variable to categorical variable and adding it to agg_df

my_labels = ['0_18', '19_23', '24_30', '31_40', '41_70']
agg_df["AGE_CUT"] = pd.cut(x=agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70], labels=my_labels)
agg_df.tail(10) # Just checking data

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CUT
338,bra,android,male,23,21.5,19_23
339,tur,android,male,21,19.0,19_23
340,tur,ios,male,47,19.0,41_70
341,bra,ios,female,34,19.0,31_40
342,bra,ios,male,47,19.0,41_70
343,usa,ios,female,38,19.0,31_40
344,usa,ios,female,30,19.0,24_30
345,can,android,female,27,19.0,24_30
346,fra,android,male,18,19.0,0_18
347,deu,android,male,26,9.0,24_30


In [ ]:
# Identify new level-based customers (Personas)
agg_df["customers_level_based"] = [f"{i[0]}_{i[1]}_{i[2]}_{i[-1]}" for i in agg_df.values]

In [ ]:
agg_df = agg_df.loc[:, ["customers_level_based", "PRICE"]].groupby("customers_level_based").agg({"PRICE": "mean"}).sort_values(by="PRICE", ascending=False).reset_index()
agg_df["customers_level_based"].head() # Just checking data again

0    fra_android_female_24_30
1          tur_ios_male_24_30
2          tur_ios_male_31_40
3    tur_android_female_31_40
4      can_android_male_19_23
Name: customers_level_based, dtype: object

In [ ]:
# Segment new customers (Personas)

agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])
agg_df.head()

,customers_level_based,PRICE,SEGMENT
0,fra_android_female_24_30,45.428571,A
1,tur_ios_male_24_30,45.000000,A
2,tur_ios_male_31_40,42.333333,A
3,tur_android_female_31_40,41.833333,A
4,can_android_male_19_23,40.111111,A


In [ ]:
# Describe the segments and especially "C"

agg_df.groupby(["SEGMENT"]).agg({"PRICE": ["mean", "max", "sum"]})

agg_df[agg_df["SEGMENT"] == "C"].describe()

,PRICE
count,27.000000
mean,33.509674
std,0.492587
min,32.500000
25%,33.000000
50%,33.627634
75%,34.000000
max,34.077340


In [ ]:
new_user = "fra_android_male_24_30"
print(agg_df[agg_df["customers_level_based"] == new_user])

     customers_level_based  PRICE SEGMENT
74  fra_android_male_24_30   33.0       C
